In [112]:
import pandas as pd
import numpy as np 
import random
from sklearn.decomposition import PCA
from tqdm import tqdm
import plotly.express as px 
import plotly.io as pio 
pio.renderers.default = "browser"

In [46]:
import random
n = 5
w = [[0.2,0.2,0.2,0.2,0.2]]
for i in range(1,30000):
    weights = [random.random() for _ in range(n)]
    sum_weights = sum(weights)
    weights = [1*w/sum_weights for w in weights]
    w.append(list(np.round(weights,2)))

seen = set()
samples_mini = [x for x in w if frozenset(x) not in seen and not seen.add(frozenset(x))]

In [47]:
len(samples_mini)

21557

In [104]:
def calculate_pdi_weights( returns,return_mean_range): 

    n = len(returns.columns)
    w = [[(100/n)/100]*n]
    for i in range(1,30000):
        weights = [random.random() for _ in range(n)]
        sum_weights = sum(weights)
        weights = [1*w/sum_weights for w in weights]
        w.append(list(np.round(weights,2)))
    seen = set()
    samples_weigth = [x for x in w if frozenset(x) not in seen and not seen.add(frozenset(x))]


    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/return_mean_range))-1
        
        return(Result)

    pca = PCA()
    PDI_dict = {}

    for y,num in tqdm(zip(samples_weigth, range(0,len(samples_weigth),1))):
        
        port_ret  = returns.mul(y,axis=1).sum(axis=1)

        ann_ret = meanRetAn(list(port_ret))
        an_cov = returns.cov()
        port_std = np.sqrt(np.dot(np.array(y).T, np.dot(an_cov, y)))*np.sqrt(return_mean_range)
        corr_matrix = np.array(returns.mul(y).cov())
        principalComponents = pca.fit(corr_matrix)
        PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1

        PDI_dict[num ] = {}
        PDI_dict[num ]["PDI_INDEX"] = PDI
        PDI_dict[num ]["# of Assets"] = len(y)
        PDI_dict[num ]["Sharpe Ratio"] = ann_ret/port_std
        PDI_dict[num ]["Annual Return"] = ann_ret
        PDI_dict[num ]["weights"] = y
        PDI_dict[num ]["Annual STD"] = port_std

    df = pd.DataFrame(PDI_dict).T
    df["PDI_INDEX"] = df["PDI_INDEX"].astype(float)
    df["Sharpe Ratio"] = df["Sharpe Ratio"].astype(float)
    df["Annual Return"] = df["Annual Return"].astype(float)
    df["Annual STD"] = df["Annual STD"].astype(float)

    return df

In [105]:
returns_weekly = pd.read_csv("weeklyReturns.csv", index_col="Date")
returns_weekly.head()

,SPY,IVV,VTI,VOO,QQQ,VEA,IEFA,AGG,VWO,IEMG,...,SMN,EFU,RXD,EEH,SBM,SZK,SIJ,SCC,FUE,LD
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-14,-0.007167,-0.007018,-0.005752,-0.007006,-0.003946,0.011404,0.008895,0.004417,0.006056,0.010519,...,0.016906,-0.017021,-0.017735,0.012709,0.020985,0.024178,0.010705,0.016117,-0.011710,-0.040179
2015-01-21,0.011053,0.010725,0.008968,0.010691,0.011688,0.028725,0.028105,0.000628,0.032104,0.025494,...,-0.051443,-0.059740,-0.018055,-0.000661,-0.034380,-0.068240,-0.020521,-0.016583,0.017773,0.083463
2015-01-28,-0.014477,-0.014572,-0.010608,-0.014713,-0.011944,-0.001566,-0.001787,0.005203,-0.003645,-0.003522,...,0.052249,0.001612,-0.001415,-0.041639,-0.005418,0.057007,0.001802,-0.050587,0.046566,-0.020510
2015-02-04,0.019586,0.019601,0.018932,0.019947,0.019322,0.017250,0.017899,0.000406,0.002195,0.006445,...,-0.069453,-0.036544,0.028329,0.040000,-0.010506,-0.005618,-0.042725,-0.030888,-0.082314,-0.017288
2015-02-11,0.014065,0.014211,0.013651,0.014106,0.018567,-0.003083,-0.000703,-0.009559,-0.016306,-0.020244,...,-0.047281,-0.001670,-0.019284,0.011273,-0.025560,-0.033145,-0.015269,-0.016733,0.000000,-0.023538


In [106]:
ini_porti = list(returns_weekly.columns[:5])
test_week = returns_weekly[ini_porti]

In [124]:
test_week

,SPY,IVV,VTI,VOO,QQQ
Date,,,,,
2015-01-14,-0.007167,-0.007018,-0.005752,-0.007006,-0.003946
2015-01-21,0.011053,0.010725,0.008968,0.010691,0.011688
2015-01-28,-0.014477,-0.014572,-0.010608,-0.014713,-0.011944
2015-02-04,0.019586,0.019601,0.018932,0.019947,0.019322
2015-02-11,0.014065,0.014211,0.013651,0.014106,0.018567
...,...,...,...,...,...
2020-12-02,0.011388,0.011291,0.009119,0.011465,0.025550
2020-12-09,0.000164,0.000489,0.005338,0.000386,-0.008359
2020-12-16,0.009050,0.008817,0.011932,0.008958,0.025456


In [107]:
df = calculate_pdi_weights( returns = test_week,return_mean_range = 52)

21660it [00:35, 608.72it/s]


In [156]:
df.iloc[0]["weights"]

[0.2, 0.2, 0.2, 0.2, 0.2]

In [117]:
for i in df["weights"][0:5]:
    print(i)

[0.2, 0.2, 0.2, 0.2, 0.2]
[0.1, 0.04, 0.38, 0.17, 0.31]
[0.42, 0.17, 0.26, 0.14, 0.01]
[0.13, 0.13, 0.36, 0.07, 0.3]
[0.22, 0.21, 0.21, 0.26, 0.1]


In [113]:
fig = px.scatter(df, x = "PDI_INDEX", y = "Annual Return")
fig.show()

In [161]:
############################################################## Trading Strategy #################################################################################
def pca_per_weights(return_data, portfolio, interval, ret_range_mean):
        data = return_data.copy() # data containing weekly returns
        tickers = list(data.columns)
        data.index = pd.to_datetime(data.index) # Conveting the index which is date to datetime
        weeks_list = data.index # grabbing all index dates
        data.index = data.index.to_period(interval) # converting the index to quarterly sets
        periods = data.index.unique() # taking the unique quarters to loop

        
        #print(periods)
        first_period = periods[0] # the first period of the time frame
        remaining_periods = periods[1:] # the remianing periods of the time framr
        first_periods = periods[:-1] # all periods minus the last

        def meanRetAn(data):             
            Result = 1
            
            for i in data:
                Result *= (1+i)
                
            Result = Result**(1/float(len(data)/ret_range_mean))-1
            
            return(Result)

        
        def port_ret(returns, period, weights): # function for calculating returns
            portfolio_weights_ew = weights
            port_return = returns.loc[period].mul(portfolio_weights_ew,axis=1).sum(axis=1)
            return  port_return

        pdi_performance_w = []
        sharpe_performance_w= []
        equal_performance = []

        
        ############################################################ Calculate first period ####################################################################################
        #### Weighted #####
        print(first_period)
        n_assets = len(portfolio) # equal weigt initialisation
        ini_weights = np.repeat(1/n_assets, n_assets) # equal weigt initialisation

        pdi_performance_w.extend(port_ret(data, portfolio,first_period,ini_weights))

        sharpe_performance_w.extend(port_ret(data, portfolio,first_period,ini_weights))

        equal_performance.extend(port_ret(data, portfolio,first_period,ini_weights))

        tickers_weekly = list(data.columns)
        #number_of_assets = [len(initial_port)]







        ######################################################## Calculation of portfolio perfomnce #############################################################################

        for init_time, next_time in zip(first_periods,remaining_periods):
            ############ Portfolio Creation ##############################
            print("first time: {} - last time: {}".format(init_time, next_time))
            PDI_DF = calculate_pdi_weights(returns = data.loc[init_time].dropna(axis=1), return_mean_range = ret_range_mean)


            ################################################## Taking the higest PDI ##########################################################################

            id = PDI_DF["PDI_INDEX"].idxmax()
            port_max_pdi_weights = PDI_DF["weights"][id]
            
            port_max_ret_period = port_ret(data, next_time,port_max_pdi_weights)
            pdi_performance_w.extend(port_max_ret_period)

            ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
            id_sharpe = PDI_DF["Sharpe Ratio"].idxmax()
            port_max_sharpe_weights = PDI_DF["weights"][id_sharpe]

            
            port_max_ret_period_sharpe = port_ret(data, next_time,port_max_sharpe_weights)
            sharpe_performance_w.extend(port_max_ret_period_sharpe)
            
            ################################################################## Equal Weigths Portoflio ##########################################################################

            equal_weights = PDI_DF.iloc[0]["weights"]

            
            port_max_ret_period_equal = port_ret(data, next_time,equal_weights)
            equal_performance.extend(port_max_ret_period_equal)




        dd = pd.DataFrame()
        dd["Time"] = weeks_list
        dd["Equal Weights"] = equal_performance
        dd["Max PDI Weights"] = pdi_performance_w
        dd["Max Sharpe Ratio Weights"] = sharpe_performance_w
        dd["Equal Weights Cummulative"] = dd["Equal Weights"].cumsum(axis=0)
        dd["Max PDI Weights Cummulative"] = dd["Max PDI Weights"].cumsum(axis=0) # cummulative returns max pdi
        dd["Max Sharpe Ratio Weights Cummulative"] = dd["Max Sharpe Ratio Weights"].cumsum(axis=0) #cummulative return sharpe ratio




        return dd

In [162]:
ini_porti = list(returns_weekly.columns[:5])
test_week = returns_weekly[ini_porti]

In [163]:
performance = pca_per_weights(return_data = test_week, portfolio = ini_porti, interval = "Q", ret_range_mean = 12)

2015Q1
first time: 2015Q1 - last time: 2015Q2
21597it [00:34, 623.94it/s]
first time: 2015Q2 - last time: 2015Q3
21656it [00:34, 636.53it/s]
first time: 2015Q3 - last time: 2015Q4
21670it [00:34, 631.65it/s]
first time: 2015Q4 - last time: 2016Q1
21722it [00:34, 633.03it/s]
first time: 2016Q1 - last time: 2016Q2
21714it [00:34, 633.50it/s]
first time: 2016Q2 - last time: 2016Q3
21666it [00:34, 629.99it/s]
first time: 2016Q3 - last time: 2016Q4
21613it [00:33, 637.29it/s]
first time: 2016Q4 - last time: 2017Q1
21710it [00:34, 625.57it/s]
first time: 2017Q1 - last time: 2017Q2
21621it [00:34, 633.41it/s]
first time: 2017Q2 - last time: 2017Q3
21520it [00:34, 622.01it/s]
first time: 2017Q3 - last time: 2017Q4
21704it [00:35, 608.28it/s]
first time: 2017Q4 - last time: 2018Q1
21646it [00:34, 626.90it/s]
first time: 2018Q1 - last time: 2018Q2
21662it [00:34, 620.52it/s]
first time: 2018Q2 - last time: 2018Q3
21582it [00:35, 615.22it/s]
first time: 2018Q3 - last time: 2018Q4
21608it [00:35, 

In [164]:
performance

,Time,Equal Weights,Max PDI Weights,Max Sharpe Ratio Weights,Equal Weights Cummulative,Max PDI Weights Cummulative,Max Sharpe Ratio Weights Cummulative
0,2015-01-14,-0.006178,-0.006178,-0.006178,-0.006178,-0.006178,-0.006178
1,2015-01-21,0.010625,0.010625,0.010625,0.004447,0.004447,0.004447
2,2015-01-28,-0.013263,-0.013263,-0.013263,-0.008816,-0.008816,-0.008816
3,2015-02-04,0.019478,0.019478,0.019478,0.010662,0.010662,0.010662
4,2015-02-11,0.014920,0.014920,0.014920,0.025582,0.025582,0.025582
...,...,...,...,...,...,...,...
303,2020-12-02,0.013763,0.012918,0.009957,0.878897,0.873213,1.066640
304,2020-12-09,-0.000397,0.000090,0.004183,0.878500,0.873304,1.070823
305,2020-12-16,0.012843,0.011954,0.011889,0.891343,0.885258,1.082712
306,2020-12-23,-0.001023,-0.001031,0.002835,0.890320,0.884227,1.085547


In [166]:
fig = px.line(performance, x="Time", y=["Max PDI Weights Cummulative","Max Sharpe Ratio Weights Cummulative","Equal Weights Cummulative"])
fig.show()